In [1]:
import torch
from torch_geometric.data import Data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv
from torch.nn import Linear, Sequential, BatchNorm1d #, ReLU
from torch.nn.functional import relu
from torch.utils.data.sampler import SubsetRandomSampler
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import to_dense_adj
from torch_geometric.nn import ARMAConv
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import global_add_pool

import argparse
import time
import math
import copy

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdmolops
from rdkit.Chem import Crippen
from rdkit.Chem import PandasTools
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.rdchem import GetPeriodicTable
from rdkit import DataStructs
from rdkit.Chem import Descriptors
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.ML.Descriptors import MoleculeDescriptors
import rdkit

from mordred import Calculator, descriptors
from padelpy import padeldescriptor, from_smiles

import xgboost as xgb

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
#from sklearn.ensemble import RandomForestClassifier as RF
from sklearn import metrics
from sklearn.svm import SVR



In [2]:
# Replace '.csv' with the actual file path
#file_path = 'D:\Research\Results\PFAS-ML\My-ML-code\My_CMC_data.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('../dataset/updated_CMC-Data-for-mixed-micelle.csv')

total_elements=len(df)
print('Total no. of smiles:', total_elements)
df.head()


Total no. of smiles: 979


,smile_A,mol_fraction_A,smile_B,mol_fraction_B,logCMC
0,CCCCCCCCCCCCOS(=O)(=O)[O-].[Na+],0.0,CCCCCCCCCCCCCCCC[N+](C)(C)C.[Br-],1.0,2.857332
1,CCCCCCCCCCCCOS(=O)(=O)[O-].[Na+],0.1,CCCCCCCCCCCCCCCC[N+](C)(C)C.[Br-],0.9,0.991226
2,CCCCCCCCCCCCOS(=O)(=O)[O-].[Na+],0.2,CCCCCCCCCCCCCCCC[N+](C)(C)C.[Br-],0.8,0.991226
3,CCCCCCCCCCCCOS(=O)(=O)[O-].[Na+],0.3,CCCCCCCCCCCCCCCC[N+](C)(C)C.[Br-],0.7,0.903090
4,CCCCCCCCCCCCOS(=O)(=O)[O-].[Na+],0.4,CCCCCCCCCCCCCCCC[N+](C)(C)C.[Br-],0.6,0.732394


In [3]:
#Create a new column with canonical format of smile notation

df['smile_A'] = [Chem.CanonSmiles(j) for j in df['smile_A']]
df['smile_B'] = [Chem.CanonSmiles(j) for j in df['smile_B']]

df.head()

,smile_A,mol_fraction_A,smile_B,mol_fraction_B,logCMC
0,CCCCCCCCCCCCOS(=O)(=O)[O-].[Na+],0.0,CCCCCCCCCCCCCCCC[N+](C)(C)C.[Br-],1.0,2.857332
1,CCCCCCCCCCCCOS(=O)(=O)[O-].[Na+],0.1,CCCCCCCCCCCCCCCC[N+](C)(C)C.[Br-],0.9,0.991226
2,CCCCCCCCCCCCOS(=O)(=O)[O-].[Na+],0.2,CCCCCCCCCCCCCCCC[N+](C)(C)C.[Br-],0.8,0.991226
3,CCCCCCCCCCCCOS(=O)(=O)[O-].[Na+],0.3,CCCCCCCCCCCCCCCC[N+](C)(C)C.[Br-],0.7,0.903090
4,CCCCCCCCCCCCOS(=O)(=O)[O-].[Na+],0.4,CCCCCCCCCCCCCCCC[N+](C)(C)C.[Br-],0.6,0.732394


In [4]:
#Removing counterions from smile strings
mol_A = [Chem.MolFromSmiles(i,sanitize=True) for i in df['smile_A']]
mol_B = [Chem.MolFromSmiles(i,sanitize=True) for i in df['smile_B']]

remover = SaltRemover(defnFormat='smarts',defnData="[Cl-,Br-,I-,Na+,Li+,K+,N]") # N is to remove NH4+ counter-ion 

count = 0
no_salt_smile_A = []
for mol in mol_A:
    stripped_mol = remover.StripMol(mol)
    temp=Chem.MolToSmiles(stripped_mol)
    #print(temp)
    no_salt_smile_A.append(temp)
    count=count+1

no_salt_smile_B = []
for mol in mol_B:
    stripped_mol = remover.StripMol(mol)
    temp=Chem.MolToSmiles(stripped_mol)
    #print(temp)
    no_salt_smile_B.append(temp)
    count=count+1


df_no_salt=df
df_no_salt['smile_A']=no_salt_smile_A
df_no_salt['smile_B']=no_salt_smile_B


Z=df_no_salt.logCMC
df_no_salt

,smile_A,mol_fraction_A,smile_B,mol_fraction_B,logCMC
0,CCCCCCCCCCCCOS(=O)(=O)[O-],0.0,CCCCCCCCCCCCCCCC[N+](C)(C)C,1.0,2.857332
1,CCCCCCCCCCCCOS(=O)(=O)[O-],0.1,CCCCCCCCCCCCCCCC[N+](C)(C)C,0.9,0.991226
2,CCCCCCCCCCCCOS(=O)(=O)[O-],0.2,CCCCCCCCCCCCCCCC[N+](C)(C)C,0.8,0.991226
3,CCCCCCCCCCCCOS(=O)(=O)[O-],0.3,CCCCCCCCCCCCCCCC[N+](C)(C)C,0.7,0.903090
4,CCCCCCCCCCCCOS(=O)(=O)[O-],0.4,CCCCCCCCCCCCCCCC[N+](C)(C)C,0.6,0.732394
...,...,...,...,...,...
974,CCCCCCCCCCCCCCOCC(O)Cn1cc[n+](CCC[n+]2ccn(CC(O...,0.5,CCCCCCCCCCCCCCOCC(O)Cn1cc[n+](CCC[n+]2ccn(CC(O...,0.5,1.851000
975,CCCCCCCCCCCCCCC(CSCCSCC(CCCCCCCCCCCCCC)[n+]1cc...,0.5,CCCCCCCCCCCCCCC(CSCCSCC(CCCCCCCCCCCCCC)[n+]1cc...,0.5,1.342000
976,CCCCCCCCCCCCCCC(CSCCCSCC(CCCCCCCCCCCCCC)[n+]1c...,0.5,CCCCCCCCCCCCCCC(CSCCCSCC(CCCCCCCCCCCCCC)[n+]1c...,0.5,1.322000
977,CCCCCCCCCCCCCCC(CSCCCCSCC(CCCCCCCCCCCCCC)[n+]1...,0.5,CCCCCCCCCCCCCCC(CSCCCCSCC(CCCCCCCCCCCCCC)[n+]1...,0.5,1.301000


In [5]:
smile_A=df_no_salt['smile_A']
smile_B=df_no_salt['smile_B']
frac_a=df_no_salt['mol_fraction_A']
frac_b=df_no_salt['mol_fraction_B']

In [137]:
# Apply maximum absolute scaling
scaled_concat_total_B_rdkit_descriptors = concat_total_B_rdkit_descriptors.copy()
for column in concat_total_B_rdkit_descriptors.columns:
    scaled_concat_total_B_rdkit_descriptors[column] = scaled_concat_total_B_rdkit_descriptors[column] / scaled_concat_total_B_rdkit_descriptors[column].abs().max()

scaled_concat_total_B_rdkit_descriptors = scaled_concat_total_B_rdkit_descriptors.fillna(0)
# View the normalized data
display(scaled_concat_total_B_rdkit_descriptors)

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,mol_fraction_B
0,0.166611,0.166611,0.142317,0.125915,0.335723,0.236941,0.166754,0.158414,0.166739,0.173789,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.151899,0.0,1.0
1,0.166611,0.166611,0.142317,0.125915,0.335723,0.236941,0.166754,0.158414,0.166739,0.173789,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.151899,0.0,0.9
2,0.166611,0.166611,0.142317,0.125915,0.335723,0.236941,0.166754,0.158414,0.166739,0.173789,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.151899,0.0,0.8
3,0.166611,0.166611,0.142317,0.125915,0.335723,0.236941,0.166754,0.158414,0.166739,0.173789,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.151899,0.0,0.7
4,0.166611,0.166611,0.142317,0.125915,0.335723,0.236941,0.166754,0.158414,0.166739,0.173789,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.151899,0.0,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,0.759042,0.759042,0.049787,-0.055154,0.072397,0.255524,0.421440,0.416281,0.421424,0.421652,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.303797,0.0,0.5
975,0.180241,0.180241,0.079508,0.070345,0.070797,0.256686,0.413301,0.408516,0.413184,0.396011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.291139,0.0,0.5
976,0.180397,0.180397,0.080522,0.071242,0.069528,0.256283,0.421521,0.416371,0.421403,0.404558,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.303797,0.0,0.5
977,0.180518,0.180518,0.081260,0.071895,0.068559,0.255896,0.429741,0.424226,0.429622,0.413105,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.316456,0.0,0.5


In [138]:
scaled_total_concat_rdkit_descriptors = pd.concat([scaled_concat_total_A_rdkit_descriptors, scaled_concat_total_B_rdkit_descriptors], axis=1)
display(scaled_total_concat_rdkit_descriptors)

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,mol_fraction_B
0,0.721452,0.721452,0.003813,-0.502686,0.399337,0.229928,0.199893,0.200050,0.199833,0.191176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.151899,0.0,1.0
1,0.721452,0.721452,0.003813,-0.502686,0.399337,0.229928,0.199893,0.200050,0.199833,0.191176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.151899,0.0,0.9
2,0.721452,0.721452,0.003813,-0.502686,0.399337,0.229928,0.199893,0.200050,0.199833,0.191176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.151899,0.0,0.8
3,0.721452,0.721452,0.003813,-0.502686,0.399337,0.229928,0.199893,0.200050,0.199833,0.191176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.151899,0.0,0.7
4,0.721452,0.721452,0.003813,-0.502686,0.399337,0.229928,0.199893,0.200050,0.199833,0.191176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.151899,0.0,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,0.745469,0.745469,0.049787,-0.055154,0.069583,0.250417,0.541659,0.530115,0.541609,0.544118,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.303797,0.0,0.5
975,0.177018,0.177018,0.079508,0.070345,0.068046,0.251555,0.531198,0.520227,0.531019,0.511029,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.291139,0.0,0.5
976,0.177171,0.177171,0.080522,0.071242,0.066826,0.251160,0.541763,0.530230,0.541582,0.522059,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.303797,0.0,0.5
977,0.177290,0.177290,0.081260,0.071895,0.065895,0.250781,0.552328,0.540234,0.552145,0.533088,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.316456,0.0,0.5


In [139]:
scaled_total_concat_rdkit_descriptors.to_csv('total_rdkit_descriptors_concat.csv', index=False)

# Generating Mordred Descriptors

In [7]:
# Workaround to handle deprecated numpy attributes
if not hasattr(np, 'float'):
    np.float = float

calc = Calculator(descriptors, ignore_3D=True)

def All_Mordred_descriptors(Smiles):
    if not hasattr(np, 'float'):
        np.float = float
        
    #calc = Calculator(descriptors, ignore_3D=True)
    
    # Calculate descriptors for each molecule and store in a list
    all_descriptors = []
    molecule_smiles = []
    
    for smi in Smiles:
        mol = Chem.MolFromSmiles(smi)
        if mol is not None:
            try:
                desc_values = calc(mol)
                descriptors = desc_values
            except Exception as e:
                print(f"Error calculating descriptors for molecule: {Chem.MolToSmiles(mol)}")
                print(f"Exception: {e}")
            
            if descriptors:
                all_descriptors.append(descriptors)
                molecule_smiles.append(smi)
        else:
            print(f"Invalid molecule: {smi}")
    
    # Convert descriptor values to a DataFrame
    if all_descriptors:
        df = pd.DataFrame(all_descriptors)
        #df.insert(0, 'SMILES', molecule_smiles)  # Add SMILES column for reference
        return df
    else:
        print("No valid descriptors were calculated.")
        return None
         

total_A_mordred_descriptors = All_Mordred_descriptors(df_no_salt['smile_A'])
total_B_mordred_descriptors = All_Mordred_descriptors(df_no_salt['smile_B'])

C:\Users\achoudh\AppData\Local\miniconda3\envs\CMC_Prediction\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [8]:
mordred_descriptor_names = [str(descriptor) for descriptor in calc.descriptors]

total_A_mordred_descriptors = pd.DataFrame(total_A_mordred_descriptors)
total_B_mordred_descriptors = pd.DataFrame(total_B_mordred_descriptors)
total_A_mordred_descriptors.columns = mordred_descriptor_names
total_B_mordred_descriptors.columns = mordred_descriptor_names

In [9]:
total_A_mordred_descriptors = total_A_mordred_descriptors.astype(float)
total_B_mordred_descriptors = total_B_mordred_descriptors.astype(float)

## Feature Merging using Arithmetic Mean - Mordred 

In [10]:
# Summation
u_mordred = [frac_a[j]*total_A_mordred_descriptors.loc[j] for j in range(len(total_A_mordred_descriptors))]
u_mordred = pd.DataFrame(u_mordred,columns=mordred_descriptor_names)
v_mordred = [frac_b[j]*total_B_mordred_descriptors.loc[j] for j in range(len(total_B_mordred_descriptors))]
v_mordred = pd.DataFrame(v_mordred,columns=mordred_descriptor_names)
total_mordred_descriptors_summation = u_mordred + v_mordred
total_mordred_descriptors_summation

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,13.911785,11.193377,0.0,1.0,23.528237,2.121318,4.242637,23.528237,1.176412,3.822931,...,8.805225,50.798349,284.331177,4.585987,1281.0,17.0,80.0,78.0,7.812500,4.875000
1,13.699653,11.078442,0.1,0.9,23.136154,2.121317,4.242634,23.136154,1.174108,3.806735,...,8.793194,50.425838,282.412849,4.758693,1230.5,16.7,78.8,76.8,7.737500,4.800000
2,13.487521,10.963508,0.2,0.8,22.744070,2.121315,4.242631,22.744070,1.171805,3.790540,...,8.781162,50.053326,280.494522,4.931398,1180.0,16.4,77.6,75.6,7.662500,4.725000
3,13.275389,10.848573,0.3,0.7,22.351987,2.121314,4.242628,22.351987,1.169501,3.774344,...,8.769130,49.680814,278.576195,5.104104,1129.5,16.1,76.4,74.4,7.587500,4.650000
4,13.063257,10.733638,0.4,0.6,21.959903,2.121313,4.242625,21.959903,1.167198,3.758149,...,8.757099,49.308302,276.657868,5.276810,1079.0,15.8,75.2,73.2,7.512500,4.575000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,36.988332,22.808782,0.0,2.0,64.486046,2.306398,4.439390,64.486046,1.264432,4.789734,...,9.968573,102.708395,718.632510,5.403252,18614.0,54.0,222.0,234.0,14.916667,12.333333
975,34.786132,24.073765,0.0,2.0,60.936393,2.314767,4.518766,60.936393,1.269508,4.731373,...,9.989161,99.564268,704.581343,5.504542,13366.0,53.0,210.0,224.0,14.166667,11.638889
976,35.493238,24.320719,0.0,2.0,62.175363,2.313120,4.513616,62.175363,1.268885,4.751267,...,10.000660,100.644293,718.596993,5.485473,14362.0,54.0,214.0,228.0,14.416667,11.888889
977,36.200345,24.564253,0.0,2.0,63.482210,2.312159,4.510329,63.482210,1.269644,4.770773,...,10.012028,101.723013,732.612644,5.467259,15407.0,55.0,218.0,232.0,14.666667,12.138889


## Scale Merged Feature Matrix 

In [11]:
# Apply maximum absolute scaling
scaled_total_mordred_descriptors_summation = total_mordred_descriptors_summation.copy()
for column in total_mordred_descriptors_summation.columns:
    scaled_total_mordred_descriptors_summation[column] = scaled_total_mordred_descriptors_summation[column] / scaled_total_mordred_descriptors_summation[column].abs().max()

scaled_total_mordred_descriptors_summation = scaled_total_mordred_descriptors_summation.fillna(0)
# View the normalized data
display(scaled_total_mordred_descriptors_summation)

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,0.164924,0.244284,0.00,0.333333,0.156907,0.776442,0.776442,0.156907,0.898910,0.681095,...,0.746002,0.294060,0.166739,0.266558,0.006764,0.094444,0.163265,0.137809,0.229592,0.164789
1,0.162409,0.241776,0.05,0.300000,0.154293,0.776442,0.776442,0.154293,0.897149,0.678210,...,0.744982,0.291904,0.165614,0.276596,0.006498,0.092778,0.160816,0.135689,0.227388,0.162254
2,0.159895,0.239267,0.10,0.266667,0.151678,0.776441,0.776441,0.151678,0.895389,0.675325,...,0.743963,0.289747,0.164489,0.286634,0.006231,0.091111,0.158367,0.133569,0.225184,0.159718
3,0.157380,0.236759,0.15,0.233333,0.149063,0.776441,0.776441,0.149063,0.893629,0.672439,...,0.742943,0.287591,0.163364,0.296673,0.005964,0.089444,0.155918,0.131449,0.222980,0.157183
4,0.154865,0.234251,0.20,0.200000,0.146448,0.776440,0.776440,0.146448,0.891869,0.669554,...,0.741924,0.285435,0.162239,0.306711,0.005698,0.087778,0.153469,0.129329,0.220776,0.154648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,0.438497,0.497778,0.00,0.666667,0.430050,0.844185,0.812450,0.430050,0.966167,0.853342,...,0.844563,0.594556,0.421424,0.314061,0.098290,0.300000,0.453061,0.413428,0.438367,0.416901
975,0.412390,0.525385,0.00,0.666667,0.406378,0.847248,0.826977,0.406378,0.970045,0.842944,...,0.846308,0.576355,0.413184,0.319948,0.070578,0.294444,0.428571,0.395760,0.416327,0.393427
976,0.420773,0.530775,0.00,0.666667,0.414641,0.846646,0.826034,0.414641,0.969569,0.846488,...,0.847282,0.582607,0.421403,0.318840,0.075837,0.300000,0.436735,0.402827,0.423673,0.401878
977,0.429155,0.536089,0.00,0.666667,0.423356,0.846294,0.825433,0.423356,0.970149,0.849963,...,0.848245,0.588852,0.429622,0.317781,0.081355,0.305556,0.444898,0.409894,0.431020,0.410329


## Saving Meged Feature (Arithmetic Mean) Matrix Mordred 

In [17]:
# Mordred
scaled_total_mordred_descriptors_summation.to_csv('total_mordred_descriptors_AM.csv', index=False)

# Saving Target Property

In [15]:
# Save to CSV (no index column)
Z.to_csv("total_property.csv", index=False, header=True)